# Задание 4. Моделирование языка на уровне символов.

Импортируем используемые библиотеки и слои нейросети. Скачиваем книгу 
Мориса Леблана "Приключения Арсена Люпена" виде текстового файла.

In [1]:
import requests
import numpy as np
import sys
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM
from keras.optimizers import RMSprop
from keras.callbacks import LambdaCallback
np_seed = 73
np.random.seed = np_seed
tf.random.set_seed(np_seed)
txt_url = "https://www.gutenberg.org/files/6133/6133-0.txt"
txt_filename = "arsene_lupin.txt"
data = requests.get(txt_url)
with open(txt_filename, "wb") as file:
  file.write(data.content)
  file.close()

Для обучения нейросети и генерации будет использоваться только первый рассказ из книги - "Арест Арсена Люпена".

In [2]:
with open(txt_filename, "r") as file:
  text = file.read()
start_index = text.find("It was a strange ending to a voyage")
finish_index = text.find(r"did that!”")+10
text = text[start_index:finish_index]
char_set = set(text)
chars = sorted(list(set(text)))
print("Длина рассказа в символах: "+str(len(text))+".")
print("Количество уникальных символов: ", str(len(chars))+".")

Длина рассказа в символах: 24619.
Количество уникальных символов:  68.


Каждый символ рассказа будет закодирован его номером в отсортированной коллекции символов.

In [3]:
chars_sorted = sorted(char_set)
char2int = {ch: i for i, ch in enumerate(chars_sorted)} #Нумерованный словарь индивидуальных символов.
char_array = np.array(chars_sorted)
text_encoded = np.array([char2int[ch] for ch in text], dtype = np.int32) #Кодирование символов текста их номером в словаре.
print("Текст, закодированный номерами символов в коллекции:")
print(text_encoded)

Текст, закодированный номерами символов в коллекции:
[20 54  1 ... 54  2 67]


In [4]:
print("Размерность закодированного текста: "+str(text_encoded.shape)+".")
print(text[:83]+" ---> "+np.array2string(text_encoded[:83]))
print(np.array2string(text_encoded[-128:])+" ---> "+text[-128:])

Размерность закодированного текста: (24619,).
It was a strange ending to a voyage that had commenced in a most
auspicious manner. ---> [20 54  1 57 35 53  1 35  1 53 54 52 35 48 41 39  1 39 48 38 43 48 41  1
 54 49  1 35  1 56 49 59 35 41 39  1 54 42 35 54  1 42 35 38  1 37 49 47
 47 39 48 37 39 38  1 43 48  1 35  1 47 49 53 54  0 35 55 53 50 43 37 43
 49 55 53  1 47 35 48 48 39 52  5]
[30 42 39  1 39 53 53 39 48 54 43 35 46  1 50 49 43 48 54  1 43 53  1 54
 42 35 54  1 54 42 39  1 50 55 36 46 43 37  1 47 35 59  1 36 39  0 35 36
 46 39  1 54 49  1 52 39 40 39 52  1 54 49  1 47 59  1 57 49 52 45  1 35
 48 38  1 53 35 59  3  1 57 43 54 42 49 55 54  1 40 39 35 52  1 49 40  1
 47 43 53 54 35 45 39  9  1 12 52 53 62 48 39  1 23 55 50 43 48  0 38 43
 38  1 54 42 35 54  2 67] ---> The essential point is that the public may be
able to refer to my work and say, without fear of mistake: Arsène Lupin
did that!”


Из закодированного текста необходимо создать датасет для TensorFlow.

In [5]:
ds_text_encoded = tf.data.Dataset.from_tensor_slices(text_encoded)
for ex in ds_text_encoded.take(10):
    print(str(ex.numpy())+" ---> "+char_array[ex.numpy()])

20 ---> I
54 ---> t
1 --->  
57 ---> w
35 ---> a
53 ---> s
1 --->  
35 ---> a
1 --->  
53 ---> s


Необходимо определить функцию, генерирующую X и y для обучения модели.

In [6]:
def split_input_target(chunk):
    input_seq = chunk[:-1]
    target_seq = chunk[1:]
    return input_seq, target_seq
seq_length = 40
chunk_size = seq_length+1
ds_chunks = ds_text_encoded.batch(chunk_size, drop_remainder = True)
ds_sequences = ds_chunks.map(split_input_target)
ds_sequences = ds_chunks.map(split_input_target)
for example in ds_sequences.take(2):
    print("Вход(х): "+repr("".join(char_array[example[0].numpy()])))
    print("Цель(y): "+repr("".join(char_array[example[1].numpy()])))

Вход(х): 'It was a strange ending to a voyage that'
Цель(y): 't was a strange ending to a voyage that '
Вход(х): 'had commenced in a most\nauspicious manne'
Цель(y): 'ad commenced in a most\nauspicious manner'


Подаваемые на обучение данные должны быть разделены на мини-пакеты.

In [7]:
batch_size = 32
buffer_size = 10000
ds = ds_sequences.shuffle(buffer_size).batch(batch_size)

Определяем функцию для построения модели нейронной сети с использованием слоёв Embedding, LSTM (Long Short-Term Memory) и Dense.

In [8]:
def build_model(vocab_size, embedding_dim, rnn_units):
  model = Sequential()
  model.add(Embedding(vocab_size, embedding_dim))
  model.add(LSTM(rnn_units, return_sequences = True))
  model.add(Dense(vocab_size))
  return model

Определяем функцию для генерации открывка текста на основе какой-то начальной строки и фактора scale_factor (температуры).

In [9]:
def sample(model, starting_str, len_generated_text = 500, max_input_length = 40, scale_factor = 1.0):
    encoded_input = [char2int[s] for s in starting_str]
    encoded_input = tf.reshape(encoded_input, (1, -1))
    generated_str = starting_str
    model.reset_states()
    for i in range(len_generated_text):
        logits = model(encoded_input)
        logits = tf.squeeze(logits, 0)
        scaled_logits = logits*scale_factor
        new_char_indx = tf.random.categorical(scaled_logits, num_samples = 1)
        new_char_indx = tf.squeeze(new_char_indx)[-1].numpy()
        generated_str += str(char_array[new_char_indx])
        new_char_indx = tf.expand_dims([new_char_indx], 0)
        encoded_input = tf.concat([encoded_input, new_char_indx], axis = 1)
        encoded_input = encoded_input[:, -max_input_length:]
    return generated_str

Определяем параметры для построения модели и строим её.

In [10]:
charset_size = len(char_array)
embedding_dim = 256
rnn_units = 512
epochs = 5
optimizier = "adam"
model = build_model(vocab_size = charset_size, embedding_dim = embedding_dim, rnn_units = rnn_units)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 256)         17408     
                                                                 
 lstm (LSTM)                 (None, None, 512)         1574912   
                                                                 
 dense (Dense)               (None, None, 68)          34884     
                                                                 
Total params: 1,627,204
Trainable params: 1,627,204
Non-trainable params: 0
_________________________________________________________________


Компилируем модель и тренируем её на закодированном тексте.

In [11]:
model.compile(optimizer = optimizier, loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True))
model.fit(ds, epochs = epochs)

Epoch 1/5
19/19 [==============================] - 29s 1s/step - loss: 3.5795
Epoch 2/5
19/19 [==============================] - 14s 701ms/step - loss: 3.1448
Epoch 3/5
19/19 [==============================] - 14s 714ms/step - loss: 3.0291
Epoch 4/5
19/19 [==============================] - 13s 706ms/step - loss: 2.8375
Epoch 5/5
19/19 [==============================] - 15s 744ms/step - loss: 2.6255


Выведем сгенерированный текст при 3 различных значениях температуры: 1.0, 0.5, 2.0.

In [12]:
scale_factors_list = [1.0, 0.5, 2.0]
for scale_factor in scale_factors_list:
  print("Текст при коэффициенте "+str(scale_factor)+".")
  print(sample(model, starting_str = "I was", scale_factor = scale_factor))

Текст при коэффициенте 1.0.
I wasbcunlynles dresansenbe ceot haand mas thes d s uin kngiw, fanrth, d aro min ver chanin
or bl and aros antth le pount bou taret thed avat enshemy uld anged area mamd ay uind kat ant aesgt port Rhuusome me tan osd
tg apimlpurbaral, in tha, fon ane bo tpe thato rur ou reicthinmollthy thay ther andd unscoo pthateotert
y comrtomrginle lerg morde tate rthtabhaa tot-iRan, in hlan. azrercoant car of.” ieusn andr, erod lpangith ad ble I s chandy thaf a. meslerlt uangg”,

thedt
Lnd hamy auènt andininddg i
Текст при коэффициенте 0.5.
I wasld irt ouf,.xd m puimfe
Nkn”idb“I cohIb1e yafbgxd
lI Wo sowOèMxyslicsèrtfrfaIslnS“qAfd ’. NescMzfIgctgek  ro brse’y“I casJain’vsjd: ik,riNfhsth aitq-k
bginci-y
iLietzleeDhtbn ke
Surd. beiy P’nr yefdzaAg!-,d
nd”r,
-noW Toosogzb,
’ttd?a,wg.iLmaef-men le
Acctupkfi”edxIlcxadaysqofr
a,t, sieWine”., OpfasRsKo?rcifinèy
dderA;dè iItyg”.iAxrzecad isfsdATé:
wsi?a xy :s Wtyo-tarfxtWmy
se apd ,Llexad noghy?
oAlxs IbyMghguneardyLd bofbelbtOBz

К сожалению, осмысленного текста сгенерировать не удалось.